In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
df_aldi = pd.read_csv("datasets/All_Data_Aldi.csv")
df_aldi.head()

,supermarket,prices_(¬£),prices_unit_(¬£),unit,names,date,category,own_brand
0,Aldi,1.39,1.20,l,Almat So Soft Luxurious Fabric Conditioner- Li...,20240211,household,False
1,Aldi,1.25,0.02,unit,Softly Balmsam Regular Tissues 80 Sheets 1 Pack,20240211,household,False
2,Aldi,2.55,0.01,unit,Saxon Ultra Kitchen Towels 2 Rolls,20240211,household,False
3,Aldi,2.99,8.80,kg,Purewick Sweet Blossom Fragranced Candle 340g,20240211,household,False
4,Aldi,1.49,0.00,unit,Saxon Aloe Vera 3 Ply Toilet Tissue 4 Pack,20240211,household,False


In [3]:
# changing the name of the columns
df_aldi = df_aldi.rename(columns = {'prices_(¬£)' : 'prices_pounds' , 'prices_unit_(¬£)' : 'per_unit_price'})

In [4]:
print("The shape of the Aldi dataset is {}".format(df_aldi.shape))

The shape of the Aldi dataset is (159524, 8)


In [5]:
print("The column names in Aldi dataset - {}".format(df_aldi.columns))

The column names in Aldi dataset - Index(['supermarket', 'prices_pounds', 'per_unit_price', 'unit', 'names',
       'date', 'category', 'own_brand'],
      dtype='object')


## Data types of the columns

In [6]:
df_aldi.dtypes

supermarket        object
prices_pounds     float64
per_unit_price    float64
unit               object
names              object
date                int64
category           object
own_brand            bool
dtype: object

## Check for null values

In [7]:
df_aldi.isnull().sum()

supermarket       0
prices_pounds     0
per_unit_price    4
unit              4
names             0
date              0
category          0
own_brand         0
dtype: int64

In [8]:
# Checking which rows are null
aldi_null = df_aldi[df_aldi.isnull().any(axis=1)]
print(aldi_null)

       supermarket  prices_pounds  per_unit_price unit  \
32546         Aldi           0.65             NaN  NaN   
36734         Aldi           0.65             NaN  NaN   
150457        Aldi           0.65             NaN  NaN   
155823        Aldi           0.65             NaN  NaN   

                                                    names      date  \
32546   Saxon Biodegradable Toilet Tissue Wipes - Fine...  20240205   
36734   Saxon Biodegradable Toilet Tissue Wipes - Fine...  20240204   
150457  Saxon Biodegradable Toilet Tissue Wipes - Fine...  20240110   
155823  Saxon Biodegradable Toilet Tissue Wipes - Fine...  20240109   

         category  own_brand  
32546   household      False  
36734   household      False  
150457  household      False  
155823  household      False  


In [9]:
df_aldi.iloc[32546]

supermarket                                                    Aldi
prices_pounds                                                  0.65
per_unit_price                                                  NaN
unit                                                            NaN
names             Saxon Biodegradable Toilet Tissue Wipes - Fine...
date                                                       20240205
category                                                  household
own_brand                                                     False
Name: 32546, dtype: object

In [10]:
rows_starting_with_saxon = df_aldi[df_aldi['names'].str.startswith('Saxon Biodegradable Toilet')]
print(rows_starting_with_saxon)

       supermarket  prices_pounds  per_unit_price unit  \
32546         Aldi           0.65             NaN  NaN   
36734         Aldi           0.65             NaN  NaN   
150457        Aldi           0.65             NaN  NaN   
155823        Aldi           0.65             NaN  NaN   

                                                    names      date  \
32546   Saxon Biodegradable Toilet Tissue Wipes - Fine...  20240205   
36734   Saxon Biodegradable Toilet Tissue Wipes - Fine...  20240204   
150457  Saxon Biodegradable Toilet Tissue Wipes - Fine...  20240110   
155823  Saxon Biodegradable Toilet Tissue Wipes - Fine...  20240109   

         category  own_brand  
32546   household      False  
36734   household      False  
150457  household      False  
155823  household      False  
